In [ ]:
import pandas as pd
import numpy as np
import io

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving topix500.csv to topix500.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['topix500.csv']), header=None, names=['Company'])

In [ ]:
df.head(10)

,Company
0,ABC-Mart Inc
1,Acom Co Ltd
2,Adeka Corp
3,Advantest Corp.
4,Aeon
5,Aeon Financial Service Co Ltd
6,Aeon Mall Co Ltd
7,AGC
8,Aica Kogyo
9,Ain Pharmaciez Inc


In [ ]:
df.shape

(497, 1)

In [ ]:
# https://www.wikidata.org/w/index.php?search=ABC-Mart+Inc&search=ABC-Mart+Inc&title=Special%3ASearch&go=Go&ns0=1&ns120=1

In [ ]:
df.insert(1, "Q Value", ["NA" for x in range (497)], True)
df.insert(2, "Suffix removed used to search", ["NA" for x in range (497)], True) #NA INDICATES RESULT NOT FOUND
df

,Company,Q Value,Suffix removed used to search
0,ABC-Mart Inc,NA,NA
1,Acom Co Ltd,NA,NA
2,Adeka Corp,NA,NA
3,Advantest Corp.,NA,NA
4,Aeon,NA,NA
...,...,...,...
492,Yoshinoya Holdings,NA,NA
493,Z Holdings,NA,NA
494,Zenkoku Hosho,NA,NA
495,Zensho Holdings Co Ltd,NA,NA


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
from urllib.parse import quote

In [ ]:
def scrape(str):
    comp = quote(str)
    url = 'https://www.wikidata.org/w/index.php?search=' + comp + '&title=Special:Search&profile=advanced&fulltext=1&advancedSearch-current=%7B%7D&ns0=1&ns120=1'
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    results = soup.findAll('span', attrs = {'class':'wb-itemlink-id'})
    return results

In [ ]:
# List of common endings which cab be removed to search
# Corp
# Co Ltd
# co Ltd.
# Corp.
# Inc
# Inc.
# Co.
# Ltd
# Co
# Ltd.
!python --version

Python 3.6.9


In [ ]:
for ind in df.index:
    company = df['Company'][ind]
    results = scrape(company)

    if len(results) > 0: # FULL NAME has a search result
      first_result = results[0].string[1:-1]
      df['Q Value'][ind] = first_result
      df['Suffix removed used to search'][ind] = ""
      continue

    if company.endswith('Corp'):
      company = company[:-4]
      df['Suffix removed used to search'][ind] = "Corp"

    elif company.endswith('Corp.'):
      company = company[:-5]
      df['Suffix removed used to search'][ind] = "Corp."

    elif company.endswith('Co'):
      company = company[:-2]
      df['Suffix removed used to search'][ind] = "Co"

    elif company.endswith('Co.'):
      company = company[:-3]
      df['Suffix removed used to search'][ind] = "Co."

    elif company.endswith('Inc'):
      company = company[:-3]
      df['Suffix removed used to search'][ind] = "Inc"

    elif company.endswith('Inc.'):
      company = company[:-4]
      df['Suffix removed used to search'][ind] = "Inc."

    elif company.endswith('Ltd'):
      company = company[:-3]
      df['Suffix removed used to search'][ind] = "Ltd"

    elif company.endswith('Ltd.'):
      company = company[:-4]
      df['Suffix removed used to search'][ind] = "Ltd."

    elif company.endswith('Co Ltd'):
      company = company[:-6]
      df['Suffix removed used to search'][ind] = "Co Ltd"

    elif company.endswith('Co Ltd.'):
      company = company[:-7]
      df['Suffix removed used to search'][ind] = "Co Ltd."


    results = scrape(company)

    if len(results) > 0: # FULL NAME has a search result
      first_result = results[0].string[1:-1]
      df['Q Value'][ind] = first_result
      continue

    else:
      print("####ERR#######")
      print(company)
      print(ind)

    # print(first_result)
#mw-content-text > div.searchresults > ul > li:nth-child(1) > div.mw-search-result-heading > a > span > span.wb-itemlink-id


####ERR#######
Aeon Financial Service Co 
5
####ERR#######
Aeon Mall Co 
6
####ERR#######
Comsys Holdings 
49
####ERR#######
Daiichikosho
61
####ERR#######
Dainippon Screen Mfg.
63
####ERR#######
Denki Kagaku Kogyo K.K.
68
####ERR#######
Hamamatsu Photonics KK
104
####ERR#######
Hoshizaki Electric
121
####ERR#######
Hoya Cor
123
####ERR#######
Jsr Cor
152
####ERR#######
Koito Mfg Co 
182
####ERR#######
Kusuri No Aoki Holdings Co 
191
####ERR#######
Mitsubishi Ufj Lease Fin 
236
####ERR#######
Murata Mfg 
249
####ERR#######
Nippon Konpo Unyu Soko
273
####ERR#######
Nippon Paint Holdings Co 
275
####ERR#######
Nishi-Nippon Fin
286
####ERR#######
Nishimatsu Const Co 
287
####ERR#######
Oracle Corp Japan
313
####ERR#######
Pan Pacific Intl
321
####ERR#######
Penta-Ocean Const Co 
324
####ERR#######
Santen Pharmaceutical Co 
345
####ERR#######
Seiko Epson Cor
354
####ERR#######
Ship Healthcare Holdings 
372
####ERR#######
Sho Bond Holdings
375
####ERR#######
Sumitomo Rubber Ind 
406
####ERR

In [ ]:
df

,Company,Q Value,Suffix removed used to search
0,ABC-Mart Inc,Q11188787,Inc
1,Acom Co Ltd,Q4674469,Ltd
2,Adeka Corp,Q6051307,Corp
3,Advantest Corp.,Q2722179,Corp.
4,Aeon,Q271413,
...,...,...,...
492,Yoshinoya Holdings,NA,NA
493,Z Holdings,Q86734122,
494,Zenkoku Hosho,NA,NA
495,Zensho Holdings Co Ltd,Q8069229,Ltd


In [ ]:
df.to_csv("TOPIX_Q.csv")